In [12]:
import os
import numpy as np
import torch
import sampling
import graph_lib
import noise_lib

from torch.utils.data import DataLoader, Dataset

from data.synthetic import utils as data_utils

import io
import PIL
import functools

torch.backends.cudnn.benchmark = True
from torch import nn, Tensor

import math
from datetime import datetime

In [13]:
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F


def transformer_timestep_embedding(timesteps, dim, device,max_period=10000):
    """
    Create sinusoidal timestep embeddings (like in transformer position encodings).
    timesteps: (batch,) or (N,)
    Returns: (batch, dim)
    """
    half = dim // 2
    freqs = torch.exp(-math.log(max_period) * torch.arange(0, half, dtype=torch.float32,device=device) / half)
    args = timesteps[:, None].float() * freqs[None]
    emb = torch.cat([torch.sin(args), torch.cos(args)], dim=-1)
    if dim % 2 == 1:  # zero pad if needed
        emb = F.pad(emb, (0, 1))
    return emb


class CatMLPScoreFunc(nn.Module):
    def __init__(self, vocab_size, seq_len,cat_embed_size, num_layers, hidden_size, time_scale_factor=1000.0):
        super().__init__()
        self.vocab_size = vocab_size
        self.cat_embed_size = cat_embed_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.time_scale_factor = time_scale_factor
        input_dim=cat_embed_size*seq_len
        self.input_dim=input_dim

        self.embed = nn.Embedding(vocab_size, cat_embed_size)
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim,hidden_size))  # will init in forward
        for _ in range(num_layers-1):
            self.layers.append(nn.Linear(hidden_size,hidden_size))  # will init in forward
        self.final = nn.Linear(hidden_size, seq_len*vocab_size)

    def forward(self, x, t):
        """
        x: (batch, seq_len) – categorical token ids
        t: (batch,) – timesteps
        """
        B, L = x.shape
        V=self.vocab_size
        x = self.embed(x)  # (B, L, cat_embed_size)
        x = x.view(B, -1)  # (B, L * cat_embed_size)

        temb = transformer_timestep_embedding(t * self.time_scale_factor, self.hidden_size,x.device).to(x.device)
        
        for layer in self.layers:
            x = layer(x) + temb
            x = F.silu(x)

        x = self.final(x)  # (B, L*vocab_size)
        x=x.view(B, L, V)
        return x


In [14]:
device = torch.device(f"cuda:{0}" if torch.cuda.is_available() else "cpu")
data_directory='data//synthetic//checkerboard'

In [ ]:
from omegaconf import OmegaConf
import utils
from model.ema import ExponentialMovingAverage
import losses
from itertools import chain
import os

graph_type='mixed' #uniform, masked, or mixed


cfg_path=f'configs//synthetic_config_{graph_type}.yaml'
cfg = OmegaConf.load(cfg_path)
cfg.model=OmegaConf.load('configs//model//tiny.yaml')
work_dir = f'for_synthetic_data_{graph_type}'
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
# Create directories for experimental logs
sample_dir = os.path.join(work_dir, "samples")
checkpoint_dir = os.path.join(work_dir, "checkpoints")
checkpoint_meta_dir = os.path.join(work_dir, "checkpoints-meta", "checkpoint.pth")
utils.makedirs(sample_dir)
utils.makedirs(checkpoint_dir)
utils.makedirs(os.path.dirname(checkpoint_meta_dir))
logger = utils.get_logger(os.path.join(work_dir, "logs"))
device = torch.device(f"cuda:{0}" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print("Found {} CUDA devices.".format(torch.cuda.device_count()))
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(
                "{} \t Memory: {:.2f}GB".format(
                    props.name, props.total_memory / (1024 ** 3)
                )
            )
else:
    print("WARNING: Using device {}".format(device))
    print(f"Found {os.cpu_count()} total number of CPUs.")

# build token graph
graph = graph_lib.get_graph(cfg, device)
    
# build score model
batch_size = cfg.training.batch_size
seq_len=32
vocab_size = 2
mask_id = vocab_size
if graph_type=='uniform':
    expand_vocab_size = vocab_size 
else:
    expand_vocab_size = vocab_size + 1
time_scale_factor=1000.0
embed_dim=256
num_layers=3

score_model = CatMLPScoreFunc(vocab_size=expand_vocab_size,cat_embed_size=embed_dim,num_layers= num_layers,
    hidden_size=embed_dim,seq_len=seq_len,time_scale_factor= 1000.0,
    ).to(device)

num_parameters = sum(p.numel() for p in score_model.parameters())
print(f"Number of parameters in the model: {num_parameters}")
ema = ExponentialMovingAverage(
        score_model.parameters(), decay=cfg.training.ema)
print(score_model)
print(f"EMA: {ema}")

# build noise
noise = noise_lib.get_noise(cfg).to(device)
#noise = DDP(noise, device_ids=[rank], static_graph=True) Z:Commented this out
sampling_eps = 1e-5


# build optimization state
optimizer = losses.get_optimizer(cfg, chain(score_model.parameters(), noise.parameters()))
print(f"Optimizer: {optimizer}")
scaler = torch.cuda.amp.GradScaler()
print(f"Scaler: {scaler}")
state = dict(optimizer=optimizer, scaler=scaler, model=score_model, noise=noise, ema=ema, step=0) 
state = utils.restore_checkpoint(checkpoint_meta_dir, state, device)


Found 1 CUDA devices.
NVIDIA GeForce GTX 1080 	 Memory: 8.00GB
Number of parameters in the model: 2245952
CatMLPScoreFunc(
  (embed): Embedding(2, 256)
  (layers): ModuleList(
    (0): Linear(in_features=8192, out_features=256, bias=True)
    (1-2): 2 x Linear(in_features=256, out_features=256, bias=True)
  )
  (final): Linear(in_features=256, out_features=64, bias=True)
)
EMA: <model.ema.ExponentialMovingAverage object at 0x00000255D7AC3F10>
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-06
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 1e-06
)
Scaler: <torch.cuda.amp.grad_scaler.GradScaler object at 0x00000255D7AC30A0>


C:\Users\bezem\AppData\Local\Temp\ipykernel_23024\1715072068.py:75: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
2025-04-18 11:21:11,060 - No checkpoint found at for_synthetic_data_uniform\checkpoints-meta\checkpoint.pth. Returned the same state as input


In [16]:
# Load the data from file - assumes we have already generated samples using generate_data.ipynb
data_file = os.path.join(data_directory, 'data.npy')
with open(data_file, 'rb') as f:
    data = np.load(f).astype(np.int64)
    print('data shape: %s' % str(data.shape))

# Define a custom Dataset to wrap the data
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = torch.from_numpy(data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Create a Dataset instance
train_set = CustomDataset(data)

# Function to cycle through DataLoader
def cycle_loader(dataloader):
    while True:
        for data in dataloader:
            yield data

# Initialize DataLoader without a sampler
train_ds = cycle_loader(DataLoader(
    train_set,
    batch_size=batch_size,
    num_workers=0,
    pin_memory=True,
    shuffle=True,  # Shuffle the data as needed
    persistent_workers=False,
))

# Create an iterator for the data
train_iter = iter(train_ds)

data shape: (10000000, 32)


In [17]:
#plotting code borrowed from Sun2023

def plot(xbin, fn_xbin2float, output_file=None):
  """Visualize binary data."""
  float_data = fn_xbin2float(xbin)
  if output_file is None:  # in-memory plot
    buf = io.BytesIO()
    data_utils.plot_samples(float_data, buf, im_size=4.1, im_fmt='png')
    buf.seek(0)
    image = np.asarray(PIL.Image.open(buf))[None, ...]
    return image
  else:
    with open(output_file, 'wb') as f:
      im_fmt = 'png' if output_file.endswith('.png') else 'pdf'
      data_utils.plot_samples(float_data, f, im_size=4.1, im_fmt=im_fmt,axis=True)


class BinarySyntheticHelper(object):
  """Binary synthetic model helper."""

  def __init__(self, seq_len,int_scale):
    self.seq_len = seq_len
    self.int_scale=int_scale
    self.bm, self.inv_bm = data_utils.get_binmap(seq_len,
                                                 'gray')

  def plot(self, xbin, output_file=None):
    fn_xbin2float = functools.partial(
        data_utils.bin2float, inv_bm=self.inv_bm,
        discrete_dim=self.seq_len, int_scale=self.int_scale)
    return plot(xbin, fn_xbin2float, output_file)

int_scale=5461.760975376213
model_helper = BinarySyntheticHelper(seq_len,int_scale)

remapping binary repr with gray code


In [18]:
#the metric used in Table 1 of Lou2023
def binary_mmd(x, y, sim_fn):
  """MMD for binary data."""
  x = x.astype(np.float32)
  y = y.astype(np.float32)
  kxx = sim_fn(x, x)
  kxx = kxx * (1 - np.eye(x.shape[0]))
  kxx = np.sum(kxx) / x.shape[0] / (x.shape[0] - 1)

  kyy = sim_fn(y, y)
  kyy = kyy * (1 - np.eye(y.shape[0]))
  kyy = np.sum(kyy) / y.shape[0] / (y.shape[0] - 1)
  kxy = np.sum(sim_fn(x, y))
  kxy = kxy / x.shape[0] / y.shape[0]
  mmd = kxx + kyy - 2 * kxy
  return mmd

def binary_exp_hamming_sim(x, y, bd):
  x = np.expand_dims(x, axis=1)
  y = np.expand_dims(y, axis=0)
  d = np.sum(np.abs(x - y), axis=-1)
  return np.exp(-bd * d)

def binary_exp_hamming_mmd(x, y, bandwidth=0.1):
  sim_fn = functools.partial(binary_exp_hamming_sim, bd=bandwidth)
  return binary_mmd(x, y, sim_fn)

In [19]:
num_train_steps=cfg.training.n_iters
log_freq=cfg.training.log_freq
snapshot_freq=cfg.training.snapshot_freq
eval_rounds=1
plot_samples=cfg.plot_samples
# Build one-step training and evaluation functions
optimize_fn = losses.optimization_manager(cfg)
train_step_fn = losses.get_step_fn(noise, graph, True, optimize_fn, cfg.training.accum)
eval_step_fn = losses.get_step_fn(noise, graph, False, optimize_fn, cfg.training.accum)
sampling_shape = (cfg.training.batch_size // (cfg.ngpus * cfg.training.accum), cfg.model.length)
sampling_fn = sampling.get_sampling_fn(cfg, graph, noise, sampling_shape, sampling_eps, device)
num_train_steps = cfg.training.n_iters
initial_step = int(state['step'])
print(f"Starting training loop at step {initial_step}.")

Starting training loop at step 0.


In [20]:
for step in range(initial_step,num_train_steps+1):
#while state['step'] < num_train_steps + 1:
    #step = state['step']
    batch=next(train_iter).to(device)
    # Compute loss using the simplified loss function
    loss=train_step_fn(state, batch)
    
        
    if step % log_freq == 0:
        print("step: %d, training_loss: %.5e" % (step, loss.mean().item()))
            
    if step > 0 and step % cfg.training.snapshot_freq == 0 or step == num_train_steps:
        # Save the checkpoint.
        save_step = step // cfg.training.snapshot_freq
        utils.save_checkpoint(os.path.join(
                        checkpoint_dir, f'checkpoint_{save_step}.pth'), state)
        
        #want to use the ema weights for sampling
        ema.store(score_model.parameters())
        ema.copy_to(score_model.parameters())

        #print the metric used in Table1 of Lou2023. Should get at least as small as 1.62e-5 to be considered done training. 
        avg_mmd = 0.0
        for i in range(eval_rounds):
            gt_data = []
            for _ in range(plot_samples // batch_size):
                gt_data.append(next(train_ds).cpu().numpy())
            gt_data = np.concatenate(gt_data, axis=0)
            gt_data = np.reshape(gt_data, (-1, seq_len))
            sample_data=[]
            for _ in range(plot_samples // batch_size):
                sample_data.append(sampling_fn(score_model).cpu().numpy())
            sample_data=np.concatenate(sample_data,axis=0)
            x0 = np.reshape(sample_data, gt_data.shape)
            mmd = binary_exp_hamming_mmd(x0, gt_data)
            avg_mmd += mmd
            model_helper.plot(x0,os.path.join(sample_dir,f'step_{save_step}_eval_round_{i}.pdf')) #plot a sample from the model

        avg_mmd = avg_mmd / eval_rounds
        print(f'Average mmd : {avg_mmd}')
        with open('output.txt', 'a') as file:
            file.write(f'time: {datetime.now()},step: {save_step}, loss: {loss.mean().item()}, MMD: {avg_mmd}\n')
            
        ema.restore(score_model.parameters())

c:\Users\bezem\Documents\REU code\model\utils.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


step: 0, training_loss: 2.42984e+01
step: 1000, training_loss: 2.35295e+01
step: 2000, training_loss: 2.08440e+01
step: 3000, training_loss: 1.99424e+01
step: 4000, training_loss: 2.07609e+01
step: 5000, training_loss: 3.32889e+01
step: 6000, training_loss: 1.94826e+01
step: 7000, training_loss: 1.96789e+01
step: 8000, training_loss: 1.98326e+01
step: 9000, training_loss: 2.04955e+01
step: 10000, training_loss: 2.72859e+01
Average mmd : 0.0001212035835974623


c:\Users\bezem\Documents\REU code\model\utils.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


step: 11000, training_loss: 2.67193e+01
step: 12000, training_loss: 1.84116e+01
step: 13000, training_loss: 2.13653e+01
step: 14000, training_loss: 2.62131e+01
step: 15000, training_loss: 2.14237e+01
step: 16000, training_loss: 2.06137e+01
step: 17000, training_loss: 1.48178e+01
step: 18000, training_loss: 2.26144e+01
step: 19000, training_loss: 2.12487e+01
step: 20000, training_loss: 2.29754e+01
Average mmd : 1.632814446689279e-05


c:\Users\bezem\Documents\REU code\model\utils.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


...

step: 271000, training_loss: 1.88365e+01
step: 272000, training_loss: 1.77788e+01
step: 273000, training_loss: 2.04835e+01
step: 274000, training_loss: 1.93604e+01
step: 275000, training_loss: 2.51989e+01
step: 276000, training_loss: 1.85279e+01
step: 277000, training_loss: 1.86710e+01
step: 278000, training_loss: 2.26047e+01
step: 279000, training_loss: 2.19829e+01
step: 280000, training_loss: 1.91292e+01
Average mmd : 2.6044790146673158e-05


c:\Users\bezem\Documents\REU code\model\utils.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


step: 281000, training_loss: 1.98236e+01
step: 282000, training_loss: 1.76718e+01
step: 283000, training_loss: 1.82537e+01
step: 284000, training_loss: 1.95154e+01
step: 285000, training_loss: 2.26115e+01
step: 286000, training_loss: 1.74093e+01
step: 287000, training_loss: 2.14619e+01
step: 288000, training_loss: 2.59510e+01
step: 289000, training_loss: 2.02672e+01
step: 290000, training_loss: 2.32667e+01
Average mmd : 2.009589560203473e-05


c:\Users\bezem\Documents\REU code\model\utils.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


step: 291000, training_loss: 1.87541e+01
step: 292000, training_loss: 2.72955e+01
step: 293000, training_loss: 3.57964e+01
step: 294000, training_loss: 2.20398e+01
step: 295000, training_loss: 1.85206e+01
step: 296000, training_loss: 1.66638e+01
step: 297000, training_loss: 1.92094e+01
step: 298000, training_loss: 1.80110e+01
step: 299000, training_loss: 2.49247e+01
step: 300000, training_loss: 1.86149e+01
Average mmd : -2.437535099542032e-06
